<a href="https://colab.research.google.com/github/sevwal/BME362_image_analysis/blob/main/Image_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **BME362: Image Processing**
We will be running a relatively simple image processing pipeline. As I'm not aware of your skill levels in coding, this script is meant to be fancy enough to excite you but simple enough to follow through easily. Of course, I will be here to help you throughout the course should you have any questions!

We will make use of the open source functionalities of Python. One powerful aspect of Python is the easy install of the abundant packages and dependencies that suit your needs. And of course: the large community that codes all of this and makes it accessible to you for free :D We will make use of:

*   `OpenCV` (Open Source Computer Vision Library, https://github.com/opencv/opencv/wiki) is a tool for computer vision and machine learning. It provides us with some powerful algorithms to process our images.
*   `Cellpose` (https://github.com/MouseLand/cellpose) is a tool for image segmentation of cells and nuclei. It provides us with the ability to train a neural network for image segmentation. Don't worry, we won't go that deep in this short training and will make use of the pre-trained models cellpose provides to us.



---



Remarks: This script was inspired by the documentation on [Cellpose](https://github.com/MouseLand/cellpose) and their example notebooks, workflows developed by Joel Lüthi ([Twitter](https://twitter.com/joel_luethi), [Github](https://github.com/jluethi)), as well as the notebook [Running cellpose 2.0 in colab with a GPU](https://colab.research.google.com/github/MouseLand/cellpose/blob/main/notebooks/run_cellpose_2.ipynb#scrollTo=Q7c7V4yEqDc_).


# **Preliminaries**

## **Step 1:** Get your files in here
Before we get started, you should select a couple of nice images that you want to take a look at. We want to make our files accessible to Google Colab. There's two ways of doing this:

1.   Connect your Google Drive, if you have one (Recommended)
2.   Upload them to Colab (Files will disappear if session disconnects)

Depending on which option you choose, uncomment the code below and make sure your images are uploaded.

In [ ]:
## Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
## Local upload
#from google.colab import files
#uploaded = files.upload()

## **Step 2:** Setup Cellpose and Import Libraries
The code below just installs OpenCV and the GPU version of Cellpose. 

After installation, you will get an error that your runtime has crashed. **Do not worry this is part of the plan**

In [4]:
#!pip install git+https://www.github.com/mouseland/cellpose.git #to install development version
!pip install cellpose
!pip install torch torchvision torchaudio

#Fix opencv error: ImportError: cannot import name '_registerMatType' from 'cv2.cv2'
!pip install "opencv-python-headless<4.3"
exit(0) #Restart Runtime to use newly installed numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 169 kB 23.0 MB/s 
     |████████████████████████████████| 4.2 MB 40.7 MB/s 
     |████████████████████████████████| 31.0 MB 1.4 MB/s 


As with all Python packages, installing is not enough. We need to import (and thus load) them to make them available for us to use.

In [9]:
#disabled installation and import of mxnet as pytorch is the default neural net
import numpy as np
import time, os, sys, random
from urllib.parse import urlparse
import skimage.io
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse
import shutil

## **Step 3:** Check GPU access and download Cellpose Models
Next up, we will download the pre-trained models from cellpose. Cellpose comes with the following pre-trained models:

*   Cytoplasm (`cyto`)
*   Cytoplasm2 (`cyto2`)
*   Cytoplasm2_Omnipose (`cyto2_omni`)
*   Bacteroa_Omnipose (`bact_omni`)
*   Nuclei (`nuclei`)

Cytoplasm and Cytoplasm2 are two differently trained cytoplasm models (remember that these models are neural networks!). We can use either one but depending on your cells one may perform better than the other. The model for nuclei is pretty solid, as nuclei shapes are very similar across cell types.

Omnipose uses a different algorithm to provide robust segmentation of elongated cell shapes (such as certain bacteria and other cells). We will not need this here.


In [ ]:
print ("Downloading Models")
from cellpose import models,core
#https://stackoverflow.com/questions/8924173/how-do-i-print-bold-text-in-python
BOLD = '\033[1m'
UNDERLINE = '\033[4m'
END = '\033[0m'

#Check if colab notebook instance has GPU access
if core.use_gpu()==False: 
  #Warnings from the ZeroCost StarDist notebook
  print(BOLD+UNDERLINE+'You do not have GPU access.'+END)
  print('Did you change your runtime ?') 
  print('If the runtime setting is correct then Google did not allocate a GPU for your session')
  print('Expect slow performance. To access GPU try reconnecting later')
  use_GPU=False
else:
  print(BOLD+UNDERLINE+"You have access to the GPU."+END+"\nDetails are:")
  print("*************************************************")
  !nvidia-smi
  use_GPU=True

print("*************************************************")
print("Libraries imported and configured")

## **Step 4:** Prepare your paths
In Step 1 you prepared your images. Now's the time to use them: enter the full path as `Input_Directory` below.

Also make sure to provide your image format as `image_format`. In general, image analysis is always only recommended for ***lossless image formats*** such as ***TIF*** or ***PNG***. Jpeg is considered a lossy format, as it deletes part of the information in your images to compress them more. For more information, Wikipedia's page on image compression is pretty good (https://en.wikipedia.org/wiki/Image_compression).

As you should have seen, we will always use naming conventions to orient ourselves in the gazillion of images we take. The simplest solution now is to just throw everything into one folder, read that into `Python` as a path, and then access the files through loops and changing pathnames. Like this, you never have to specify individual files, which is helpful if you're dealing with >10k images.

In [34]:
#Specify input directory
Input_Directory = "/content/gdrive/MyDrive/BME362_sample_data"
input_dir = os.path.join(Input_Directory, "") #adds separator to the end regardless if path has it or not

#Specify img format
image_format = "png"

#Create a new folder where we will store our mask
save_dir = input_dir+"Masks/"
if not os.path.exists(save_dir):
  os.makedirs(save_dir)
else:
  #If there already is a folder, delete it
  print("Existing Mask Directory found. Deleting it.")
  shutil.rmtree(save_dir)

# r=root, d=directories, f = files
files=[]

for r, d, f in os.walk(input_dir):
    for fil in f:
      if (image_format):
        if fil.endswith(image_format):
          files.append(os.path.join(r, fil))
      else:
        files.append(os.path.join(r, fil))
    break #only read the root directory; can change this to include levels
if(len(files)==0):
  print("Number of images loaded: %d." %(len(files)))
  print("Cannot read image files. Check if folder has images")
else:
  print("Number of images loaded: %d." %(len(files)))

/content/drive/MyDrive/BME362_sample_data/AssayPlate_Greiner_655090_D02_T0001F001L01A01Z01C01.png


# **On to our images!**

## **Step 5**: Read and Load all the images
In this step we will check our images. As always, we would like to check if everything is set up correctly before blindly heading into later steps. So this will allow us to see if we are working with the correct images and double check the channel names for later.

Note that *we do not need to do anything here*, we can simply run this code as we have specified all necessary variables already. ٩(◕‿◕)۶

In [31]:
imgs=[] #store all images
#Read images
for f in files:
  im=skimage.io.imread(f)
  n_dim=len(im.shape) #shape of image
  dim=im.shape #dimensions of image
  channel=min(dim) #channel will be dimension with min value usually
  channel_position=dim.index(channel)
  #if no of dim is 3 and channel is first index, swap channel to last index
  if n_dim==3 and channel_position==0: 
    #print(dim)
    im=im.transpose(1,2,0)
    dim=im.shape
    #print("Shape changed")
  #print(dim)
  imgs.append(im)

nimg = len(imgs)
print("No of images loaded are: ", nimg)

print("Example Image:")

random_idx = random.choice(range(len(imgs)))
x=imgs[random_idx]
n_dim=len(x.shape)
file_name=os.path.basename(files[random_idx])
print(file_name+" has "+str(n_dim)+" dimensions/s")
if n_dim==3:
  channel_image=x.shape[2]
  fig, axs = plt.subplots(1, channel_image,figsize=(12,5))
  print("Image: %s" %(file_name))
  for channel in range(channel_image):
      axs[channel].imshow(x[:,:,channel])
      axs[channel].set_title('Channel '+str(channel+1),size=5)
      axs[channel].axis('off')
  fig.tight_layout()
elif n_dim==2:
  print("One Channel")
  plt.imshow(x)
else:
  print("Channel number invalid or dimensions wrong. Image shape is: "+str(x.shape))

## **Step 6:** Specify parameters for the nuclear segmentation

In [1]:
model_choice = "Cytoplasm2" #@param ["Cytoplasm","Cytoplasm2", "Cytoplasm2_Omnipose", "Bacteria_Omnipose", "Nuclei"]

print("Using model ",model_choice)

Channel_for_segmentation="2" #@param[0,1,2,3]
segment_channel=int(Channel_for_segmentation)

Use_nuclear_channel=True
Nuclear_channel="1"
nuclear_channel=int(Nuclear_channel)

diameter=0

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

model_type="nuclei"

# channels = [cytoplasm, nucleus]
if model_choice not in "Nucleus":
  if Use_nuclear_channel:
    channels=[segment_channel,nuclear_channel]
  else:
    channels=[segment_channel,0]
else: #nucleus
  channels=[segment_channel,0]


# DEFINE CELLPOSE MODEL
# model_type='cyto' or model_type='nuclei'
model = models.Cellpose(gpu=use_GPU, model_type=model_type)

# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
# channels = [[2,3], [0,0], [0,0]]

# if diameter is set to None, the size of the cells is estimated on a per image basis
# you can set the average cell `diameter` in pixels yourself (recommended) 
# diameter can be a list or a single number for all images
if diameter is 0:
  diameter = None
  print("Diameter is set to None. The size of the cells will be estimated on a per image basis")

Using model  Cytoplasm2


NameError: ignored

## **Step 7:** Test the nuclear segmentation
Next up, we want to check our images and have a look at what we have. For this,we will create a quick function that displays three of our images using `matplotlib.pyplot` (imported as `plt`).

In [47]:
from skimage.util import img_as_ubyte

Image_Number = 1 #indexing starts at zero
#print(Image_Number)
diameter = 400
flow_threshold = 0.4

Cell_Probability_Threshold=0
cellprob_threshold=Cell_Probability_Threshold

if diameter is 0:
  diameter = None
if Image_Number is -1:
  Image_Number=0
  #print("Image_Number is set to zero, opening first image.")
try:
    image = imgs[Image_Number]
except IndexError as i:
   print("Image number does not exist",i)
   print("Actual no of images in folder: ",len(imgs))
print("Image: %s" %(os.path.splitext(os.path.basename(files[Image_Number]))[0]))
img1=imgs[Image_Number]

import cv2

masks, flows, styles, diams = model.eval(img1, resample=True,
                                         diameter=diameter, 
                                         flow_threshold=flow_threshold,
                                         cellprob_threshold=cellprob_threshold, 
                                         channels=channels)



# DISPLAY RESULTS
from cellpose import plot
maski = masks
flowi = flows[0]

#convert to 8-bit if not so it can display properly in the graph
if img1.dtype!='uint8':
  img1=img_as_ubyte(img1)

fig = plt.figure(figsize=(24,8))
plot.show_segmentation(fig, img1, maski, flowi, channels=channels)
plt.tight_layout()
plt.show()

## **Step 8:** Specify parameters for the cytoplasmic segmentation

## **Step 9:** Test the cytoplasmic segmentation

## **Step 10:** Run cellpose, segment cells, and write output to folder

In [ ]:
#@markdown ### **Step 8. Run Cellpose on folder of images**

#@markdown ###Tick if you want to save the flow image/s: 
Save_Flow= False #@param {type:"boolean"}
#@markdown ##### *Flow image will be resized when saved
save_flow=Save_Flow

print("Running segmentation on channel %s" %(segment_channel))
print("Using the model: ",model_choice)
if diameter is None:
  print("Diameter will be estimated from the image/s")
else:
  print(f"Cellpose will use a diameter of {diameter}")

print(f"Using a flow threshold of: {flow_threshold} and a cell probability threshold of: {cellprob_threshold}")

#if too many images, it will lead to memory error. 
#will evaluate on a per image basis
#masks, flows, styles, diams = model.eval(imgs, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)


#save images in folder with the diameter value used in cellpose
print("Segmentation Done. Saving Masks and flows now")
print("Save Directory is: ",save_dir)
if (not os.path.exists(save_dir)):
    os.mkdir(save_dir)

if save_flow:
  print("Saving Flow")
  flows_save_dir=save_dir+"flows"+os.sep
  print("Save Directory for flows is: ",flows_save_dir)
  if (not os.path.exists(flows_save_dir)):
      os.mkdir(flows_save_dir)


for img_idx, img in enumerate(imgs):
    file_name=os.path.splitext(os.path.basename(files[img_idx]))[0]
    print("\nSegmenting: ",file_name)
    mask, flow, style, diam = model.eval(img, diameter=diameter, flow_threshold=flow_threshold,cellprob_threshold=cellprob_threshold, channels=channels)
    #save images in folder with the diameter value used in cellpose
    print("Segmentation complete . Saving Masks and flows")
    #Output name for masks
    mask_output_name=save_dir+"MASK_"+file_name+".tif"
    #Save mask as 16-bit in case this has to be used for detecting than 255 objects
    mask=mask.astype(np.uint16)
    #Save flow as 8-bit
    skimage.io.imsave(mask_output_name,mask, check_contrast=False)
    if save_flow:
      #Output name for flows
      flow_output_name=flows_save_dir+"FLOWS_"+file_name+".tif"
      #Save as 8-bit
      flow_image=flow[0].astype(np.uint8)
      skimage.io.imsave(flow_output_name,flow_image, check_contrast=False)

#Save parameters used in Cellpose
parameters_file=save_dir+"Cellpose_parameters_used.txt" 
outFile=open(parameters_file, "w") 
outFile.write("CELLPOSE PARAMETERS\n") 
outFile.write("Model: "+model_choice+"\n") 
if diameter == 0:
  diameter = "Automatically estimated by cellpose"
outFile.write("Omni Flag: "+str(omni)+"\n") 
outFile.write("Diameter: "+str(diameter)+"\n") 
outFile.write("Flow Threshold: "+str(flow_threshold)+"\n") 
outFile.write("Cell probability Threshold: "+str(cellprob_threshold)+"\n") 
outFile.close() 
print("\nSegmentation complete and files saved")